# 목표: 시각화를 통해 후처리 방안을 모색하고, 모델별 예측 결과를 비교하자

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import json

## 여기 일단 skip

In [3]:
# klue mrc 데이터 ?
df = pd.read_csv("/opt/ml/mrc-level2-nlp-05-1031/outputs/all_predictions/klue-mrc-v1.1_dev.csv")
df.head()

,title,context,question,answers,guid
0,식품 알레르기,식품 알레르기(食品 Allergie)는 꽃가루나 향생제의 알레르기와는 달리 특정 사...,알레르기를 관리하는 계획에는 어떤 호르몬을 투여하는 것도 포함되는가?,['아드레날린'],klue-mrc-v1_dev_00518
1,일년전쟁,"전시 조약에 그쳤던 남극 조약의 체결에 의해 단기 결전에 의한 전쟁 종결,독립 자치...",연방군의 레이다를 무력화시킨 화학무기는?,['미노프스키 입자'],klue-mrc-v1_dev_00972
2,모나 색스,《맥스 페인 2》에서 모나의 시각 배우는 할리우드 모델 캐시 통이다 첫 두 게임의 ...,"본인의 일에 진중하게 임했고 많은 것을 연습했지만, 자신의 옷차림에 불만을 제기했던...",['쿠니스'],klue-mrc-v1_dev_00256
3,이규완,한일 합방\n\n1910년(융희 4년) 4월 29일 공립 춘천실업학교가 개교하자 강...,이규완이 강원도에 철도 건설을 제안할 당시 직책은 무엇이었는가?,['강원도도장관'],klue-mrc-v1_dev_01517
4,백야,"대표적으로 백야가 일어나는 장소는 노르웨이의 노르카프라는 도시이며, 이곳은 여름철일...",노르웨이 노르카프에서는 어느 계절에 백야를 항상 볼 수 있나요?,"['여름철', '여름']",klue-mrc-v1_dev_00666


In [6]:
guid = df["guid"]
giud_ids = [i[-5:] for i in guid]
print(giud_ids)

['00518', '00972', '00256', '01517', '00666', '00536', '01554', '01316', '01239', '01689', '01485', '00013', '01775', '01721', '01624', '01690', '01543', '00021', '00791', '00112', '00086', '01582', '00841', '00627', '01634', '01466', '00727', '01692', '01436', '01011', '01310', '00225', '00008', '00691', '01280', '01179', '00181', '01727', '00376', '00788', '00292', '00862', '01229', '01214', '00651', '01797', '01242', '00908', '01802', '01155', '00715', '01729', '01829', '00695', '01735', '00769', '01332', '00521', '01635', '01516', '01113', '01555', '00623', '00235', '01619', '01594', '00094', '00582', '00550', '00221', '00650', '00954', '00780', '01394', '01269', '01545', '01141', '01301', '00227', '01020', '01541', '01425', '00870', '01670', '01603', '00993', '01027', '00116', '00653', '00713', '01443', '00143', '01662', '01277', '00621', '00320', '01426', '01724', '00690', '00580', '01370', '01592', '00694', '01600', '00573', '00267', '00218', '01457', '01272', '01523', '01549', 

In [20]:
# EM60 prediction
df_2 = pd.read_json("/opt/ml/mrc-level2-nlp-05-1031/outputs/all_predictions/EM-60-000.json", typ='dictionary')
df_2.head()
# guid_2 = df_2["guid"]
# giud_ids_2 = [i[-5:] for i in guid_2]

# print(giud_ids_2)
# type(df_2)

mrc-1-000653          네페로스 행성
mrc-1-001113               냉전
mrc-0-002191    대통령인 빌헬름 미클라스
mrc-0-003951             뉴질랜드
mrc-1-001272              프랑스
dtype: object

In [4]:
import json
with open('/opt/ml/mrc-level2-nlp-05-1031/outputs/all_predictions/EM-60-000-val.json', 'r') as f:
    data = json.load(f)
pred_df = pd.DataFrame({'prediction': data})

In [5]:
pred_df = pred_df.reset_index().rename(columns={"index":"id"})
pred_df.head()

,id,prediction
0,mrc-0-000118,베이징
1,mrc-0-000159,일등병 토머스
2,mrc-0-000166,경위대식 망원경
3,mrc-0-000215,스위스
4,mrc-0-000223,매년 음력 정월


## 여기까지 일단 skip

## validation 240개 가지고 비교하자

In [6]:
from datasets import load_from_disk
datasets = load_from_disk("/opt/ml/data/train_dataset")
val_dataset = datasets["validation"]

validation dataset을 살펴보자.

In [7]:
val_dataset = pd.DataFrame(val_dataset)
val_dataset.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184


다행히 중복 데이터는 없다.

In [8]:
# val_dataset.info()
val_dataset["answers"].nunique

<bound method IndexOpsMixin.nunique of 0              {'answer_start': [284], 'text': ['한보철강']}
1             {'answer_start': [146], 'text': ['1871년']}
2               {'answer_start': [517], 'text': ['나뭇잎']}
3              {'answer_start': [1109], 'text': ['금대야']}
4            {'answer_start': [386], 'text': ['수평적 관계']}
                             ...                        
235           {'answer_start': [1084], 'text': ['제 양왕']}
236    {'answer_start': [343], 'text': [''일곱 개의 신전 광장...
237          {'answer_start': [247], 'text': ['미나미 지로']}
238        {'answer_start': [563], 'text': ['200,000명']}
239            {'answer_start': [811], 'text': ['공산주의']}
Name: answers, Length: 240, dtype: object>

answer text만 뽑기 위해서 column 확장

In [9]:
# "answers" column 확장
answer_start_list = []
answer_text_list = []

for _, row in val_dataset.iterrows():
    answer_start_list.append(row.answers['answer_start'][0])
    answer_text_list.append(row.answers['text'][0])

val_dataset['answer_start'] = answer_start_list
val_dataset['answer_text'] = answer_text_list
val_dataset.head()

,title,context,question,id,answers,document_id,__index_level_0__,answer_start,answer_text
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146,284,한보철강
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106,146,1871년
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180,517,나뭇잎
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141,1109,금대야
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184,386,수평적 관계


In [10]:
target_df = val_dataset[["id", "answer_text"]]
target_df.head()

,id,answer_text
0,mrc-0-003264,한보철강
1,mrc-0-004762,1871년
2,mrc-1-001810,나뭇잎
3,mrc-1-000219,금대야
4,mrc-1-000285,수평적 관계


이제 모델 예측 df를 구하자

In [11]:
with open('/opt/ml/mrc-level2-nlp-05-1031/outputs/all_predictions/EM-60-000-val.json', 'r') as f:
    data = json.load(f)
pred_df = pd.DataFrame({'prediction': data})

In [12]:
pred_df = pred_df.reset_index().rename(columns={"index":"id"})


In [14]:
display(target_df.head())
display(pred_df.head())


,id,answer_text
0,mrc-0-003264,한보철강
1,mrc-0-004762,1871년
2,mrc-1-001810,나뭇잎
3,mrc-1-000219,금대야
4,mrc-1-000285,수평적 관계


,id,prediction
0,mrc-0-000118,베이징
1,mrc-0-000159,일등병 토머스
2,mrc-0-000166,경위대식 망원경
3,mrc-0-000215,스위스
4,mrc-0-000223,매년 음력 정월


# scatter plot: 가로축은 모델/실험 & 세로축은 240개 id

In [ ]:
# load target & prediction


# make bool list of 240 ids
# or index number b/c scatter plot requires values